# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [28]:
# import libraries
import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession

from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.types import TimestampType


from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, desc, countDistinct, first
from pyspark.sql.functions import udf
from pyspark.sql.functions import to_date, year, month, dayofmonth, dayofweek, hour, date_format, substring,datediff
from pyspark.sql.functions import sum as spark_sum, avg as spark_avg



import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns


In [29]:
# create a Spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify Data Exploration") \
    .getOrCreate()

In [30]:
spark.sparkContext.getConf().getAll()

[('spark.app.id', 'local-1700172386471'),
 ('spark.driver.port', '32841'),
 ('spark.driver.host',
  '5863f353be737fc38d9029e58da34b2002b1ee70-6856f894f4-kkj8j'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.app.name', 'Sparkify Data Exploration'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true')]

In [31]:
spark

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [32]:
path = "mini_sparkify_event_data.json"
user_log = spark.read.json(path)

In [33]:
# printing out header
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [34]:
# print out the first record
user_log.take(1)

[Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30')]

It looks like one row is a record of a song played, so we have the artist name and the song name, as well as the length of the session. 

In [35]:
#user_log.filter(col("page") == "Cancel").take(1)

When the user is in a page without song playing, length it None, so length correspond to the length of the song played.

## Missing data and invalid data

We will start exploring the data using SQL, so we need to create a temporary view.

In [36]:
user_log.createOrReplaceTempView("user_log_table")
spark.sql('DESC user_log_table\
          ').show()

+-------------+---------+-------+
|     col_name|data_type|comment|
+-------------+---------+-------+
|       artist|   string|   null|
|         auth|   string|   null|
|    firstName|   string|   null|
|       gender|   string|   null|
|itemInSession|   bigint|   null|
|     lastName|   string|   null|
|       length|   double|   null|
|        level|   string|   null|
|     location|   string|   null|
|       method|   string|   null|
|         page|   string|   null|
| registration|   bigint|   null|
|    sessionId|   bigint|   null|
|         song|   string|   null|
|       status|   bigint|   null|
|           ts|   bigint|   null|
|    userAgent|   string|   null|
|       userId|   string|   null|
+-------------+---------+-------+



As suggested let's see if userId has any missing data or sessionId

In [37]:
### Missing or invalid data in userids or sessionid
spark.sql('SELECT * \
           FROM user_log_table \
           WHERE userId IS NULL \
            ').show()

+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+---+---------+------+
|artist|auth|firstName|gender|itemInSession|lastName|length|level|location|method|page|registration|sessionId|song|status| ts|userAgent|userId|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+---+---------+------+
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+---+---------+------+



In [38]:
spark.sql('SELECT * \
           FROM user_log_table \
           WHERE sessionId IS NULL \
            ').show()

+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+---+---------+------+
|artist|auth|firstName|gender|itemInSession|lastName|length|level|location|method|page|registration|sessionId|song|status| ts|userAgent|userId|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+---+---------+------+
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+---+---------+------+



There is no null values for both userId or sessionId. Maybe missing data is coded differently. Let's look at the userId list.

In [39]:
spark.sql('SELECT DISTINCT userId \
           FROM user_log_table \
           ORDER BY userId \
            ').show()

+------+
|userId|
+------+
|      |
|    10|
|   100|
|100001|
|100002|
|100003|
|100004|
|100005|
|100006|
|100007|
|100008|
|100009|
|100010|
|100011|
|100012|
|100013|
|100014|
|100015|
|100016|
|100017|
+------+
only showing top 20 rows



In [40]:
spark.sql('SELECT DISTINCT sessionId \
           FROM user_log_table \
           ORDER BY sessionId \
            ').show()

+---------+
|sessionId|
+---------+
|        1|
|        2|
|        3|
|        4|
|        5|
|        6|
|        7|
|        8|
|        9|
|       10|
|       11|
|       12|
|       13|
|       15|
|       16|
|       17|
|       18|
|       19|
|       20|
|       21|
+---------+
only showing top 20 rows



There are userids that have empty strings. Who are these users?
Since we don't know if this is the same user or multiple users, we are going to remove all userid with empty strings. We know that in auth, there is a guest value, let's check if the empty string userid are those who are guest so they did not register to the platform.

In [41]:
spark.sql('SELECT COUNT(userId) AS empty_strings_in_userid \
           FROM user_log_table \
           WHERE userId = "" \
            ').show()

+-----------------------+
|empty_strings_in_userid|
+-----------------------+
|                   8346|
+-----------------------+



In [42]:
spark.sql('SELECT DISTINCT auth\
           FROM user_log_table \
           WHERE userId = "" \
          ').show()

+----------+
|      auth|
+----------+
|Logged Out|
|     Guest|
+----------+



We see that both logged out and guests do not have a userId, so it makes sense to remove them, since we cannot associated them with any user.

In [43]:
#filtering out rows with '' for userId
condition = user_log.userId != ''
user_log_filtered= user_log.filter(condition)

In [45]:
#rebuild database view
user_log_filtered.createOrReplaceTempView("user_log_table_filtered")


In [46]:
spark.sql('SELECT COUNT(*) num_records_after_filtering\
           FROM user_log_table_filtered\
           ').show()

+---------------------------+
|num_records_after_filtering|
+---------------------------+
|                     278154|
+---------------------------+



In [47]:
spark.sql('SELECT COUNT(DISTINCT userId) AS userId_count\
           FROM user_log_table_filtered\
           ').show()

+------------+
|userId_count|
+------------+
|         225|
+------------+



We have 278154 remaining records and 225 users. (which makes sense since userid with empty strings counted as 1).

Let's continue with the user_log_filtered df

In [50]:
spark.sql('SELECT page, COUNT(*) AS number_visits\
            FROM user_log_table_filtered\
            GROUP BY page\
            ORDER BY page').show()

+--------------------+-------------+
|                page|number_visits|
+--------------------+-------------+
|               About|          495|
|          Add Friend|         4277|
|     Add to Playlist|         6526|
|              Cancel|           52|
|Cancellation Conf...|           52|
|           Downgrade|         2055|
|               Error|          252|
|                Help|         1454|
|                Home|        10082|
|              Logout|         3226|
|            NextSong|       228108|
|         Roll Advert|         3933|
|       Save Settings|          310|
|            Settings|         1514|
|    Submit Downgrade|           63|
|      Submit Upgrade|          159|
|         Thumbs Down|         2546|
|           Thumbs Up|        12551|
|             Upgrade|          499|
+--------------------+-------------+



In [49]:
#checking unique values in auth
spark.sql('SELECT DISTINCT auth, COUNT (DISTINCT userId) AS count\
            FROM user_log_table_filtered \
            GROUP BY auth\
            ').show()
#auth show the status of the user when they are using the platform
#NB: check the number of unique users using the platform as cancelled and see if it matched with churn users

+---------+-----+
|     auth|count|
+---------+-----+
|Cancelled|   52|
|Logged In|  225|
+---------+-----+



This look right, there are a total of 225 unique users who at some point had auth= logged in and the 52 users among these who cancelled.
Since this will represent churn too, I will drop auth.

In [11]:
#checking unique values in status
spark.sql('SELECT DISTINCT status\
            FROM user_log_table\
            ').show()


+------+
|status|
+------+
|   307|
|   404|
|   200|
+------+



In [12]:
#checking unique values in method
spark.sql('SELECT DISTINCT method\
            FROM user_log_table\
            ').show()


+------+
|method|
+------+
|   PUT|
|   GET|
+------+



In [13]:
spark.sql('SELECT DISTINCT page\
          FROM user_log_table').show()

+--------------------+
|                page|
+--------------------+
|              Cancel|
|    Submit Downgrade|
|         Thumbs Down|
|                Home|
|           Downgrade|
|         Roll Advert|
|              Logout|
|       Save Settings|
|Cancellation Conf...|
|               About|
| Submit Registration|
|            Settings|
|               Login|
|            Register|
|     Add to Playlist|
|          Add Friend|
|            NextSong|
|           Thumbs Up|
|                Help|
|             Upgrade|
+--------------------+
only showing top 20 rows



In [14]:
#How many records does the dataset have?
spark.sql('SELECT COUNT(*) AS number_rows\
            FROM user_log_table').show()

+-----------+
|number_rows|
+-----------+
|     286500|
+-----------+



In [15]:
#How many distint users?
spark.sql('SELECT COUNT(DISTINCT userid) AS unique_users\
            FROM user_log_table\
            ').show()

+------------+
|unique_users|
+------------+
|         226|
+------------+



In this dataset there are 226 unique users

In [16]:
# how many paid and free users?
spark.sql('SELECT COUNT(DISTINCT userid) AS paid_free\
            FROM user_log_table\
            GROUP BY level').show()

+---------+
|paid_free|
+---------+
|      196|
|      166|
+---------+



This doesn't sum up so it means that at some point, some users went from paid to free, or vice versa

Let's take a look at page visits

The churn column will be create using Cancellation Confirmation, which is the same number as Cancel. We can quickly check if the userId who went on cancel are the same as those who went to Cancellation confirmation

#### Feature documentation and decision 

The data includes the following columns:
- artist (string) : artist/singer name - keep
- auth (string): user status - **drop as redundant with churn**
- firstName (string) : user first name - **drop as redundant with userid**
- gender (string): user's gender (female or male) - keep - can be changed to categorical
- itemInSession (long): number of events during the current session - keep
- lastName (string) : user's lastname - **drop as redundant with userid**
- length (double): duration of song played - keep 
- level (string): subscription level (paid or free) - keep - can be changed to categorical.
- location (string): user's location format city, state (in the USA) - keep- can be engineered by separating state
- method (string): (put or get) - **drop**
- page (string): page where the user is - keep
- registration (long): timestamp of when the user registered. - keep - change time format, and check if there is any data for guests.
- sessionId (long): unique Id for the session the user is in ***drop***
- song (string): song playing - keep
- status (long): looks like page error status
- ts (long): timestamp , must be time when the user started the session 
- userAgent ( string ) : browser keep and change to categorical
- userId ( string) unique Id per user - keep 

#### Features to drop
- firstname, lastname, method, auth

In [18]:
cols = ("firstname","lastname","method","auth")
user_log = user_log.drop(*cols)
user_log.printSchema()  

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [19]:
user_log.createOrReplaceTempView("user_log_table")
spark.sql('DESC user_log_table\
          ').show()

+-------------+---------+-------+
|     col_name|data_type|comment|
+-------------+---------+-------+
|       artist|   string|   null|
|         auth|   string|   null|
|       gender|   string|   null|
|itemInSession|   bigint|   null|
|       length|   double|   null|
|        level|   string|   null|
|     location|   string|   null|
|         page|   string|   null|
| registration|   bigint|   null|
|    sessionId|   bigint|   null|
|         song|   string|   null|
|       status|   bigint|   null|
|           ts|   bigint|   null|
|    userAgent|   string|   null|
|       userId|   string|   null|
+-------------+---------+-------+



# Feature engineering
This will include:
- taking care of any time related features: ts and registration
- creating new features using time related features: registration month etc..
- identifying missing data and make a decision on these
- finalizing features selected and check their type (categorical binary etc.)

## Reformating time features into date

In [20]:
# these two columns have timestamps so they will be engineered to extract data on time (day, month, year)
spark.sql('SELECT ts, registration \
          FROM user_log_table\
          LIMIT 10 ').show()

+-------------+-------------+
|           ts| registration|
+-------------+-------------+
|1538352117000|1538173362000|
|1538352180000|1538331630000|
|1538352394000|1538173362000|
|1538352416000|1538331630000|
|1538352676000|1538173362000|
|1538352678000|1538331630000|
|1538352886000|1538331630000|
|1538352899000|1538173362000|
|1538352905000|1538173362000|
|1538353084000|1538173362000|
+-------------+-------------+



ts and registration are timestamps so we will convert into readable time

In [21]:
user_log = user_log.withColumn('ts_ts', (col('ts') / 1000.0).cast(TimestampType()))

In [22]:
user_log = user_log.withColumn('ts_todate', to_date('ts_ts'))

In [23]:
user_log = user_log.withColumn('registration_ts', (col('registration') / 1000.0).cast(TimestampType()))

In [24]:
user_log = user_log.withColumn('registration_todate', to_date('registration_ts'))

In [25]:
cols = ("ts","registration","ts_ts","registration_ts")
user_log = user_log.drop(*cols)

In [26]:
#recreate sql view
user_log.createOrReplaceTempView("user_log_table")
spark.sql('DESC user_log_table').show()

+-------------------+---------+-------+
|           col_name|data_type|comment|
+-------------------+---------+-------+
|             artist|   string|   null|
|               auth|   string|   null|
|             gender|   string|   null|
|      itemInSession|   bigint|   null|
|             length|   double|   null|
|              level|   string|   null|
|           location|   string|   null|
|               page|   string|   null|
|          sessionId|   bigint|   null|
|               song|   string|   null|
|             status|   bigint|   null|
|          userAgent|   string|   null|
|             userId|   string|   null|
|          ts_todate|     date|   null|
|registration_todate|     date|   null|
+-------------------+---------+-------+



#### ts and registration were reformated and are now dates. Let's take a look

In [27]:
spark.sql('SELECT ts_todate, registration_todate\
           FROM user_log_table').show()

+----------+-------------------+
| ts_todate|registration_todate|
+----------+-------------------+
|2018-10-01|         2018-09-28|
|2018-10-01|         2018-09-30|
|2018-10-01|         2018-09-28|
|2018-10-01|         2018-09-30|
|2018-10-01|         2018-09-28|
|2018-10-01|         2018-09-30|
|2018-10-01|         2018-09-30|
|2018-10-01|         2018-09-28|
|2018-10-01|         2018-09-28|
|2018-10-01|         2018-09-28|
|2018-10-01|         2018-09-30|
|2018-10-01|         2018-09-30|
|2018-10-01|         2018-09-28|
|2018-10-01|         2018-09-30|
|2018-10-01|         2018-09-30|
|2018-10-01|         2018-09-28|
|2018-10-01|         2018-09-30|
|2018-10-01|         2018-09-19|
|2018-10-01|         2018-09-28|
|2018-10-01|         2018-09-30|
+----------+-------------------+
only showing top 20 rows



In [ ]:
# what time interval are the records of this subset?
spark.sql('SELECT MIN(ts_todate), MAX(ts_todate)\
           FROM user_log_table_filtered\
          ').show()

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

In [ ]:
spark.sql('SELECT COUNT(DISTINCT userid)\
           FROM user_log_table_filtered\
           WHERE page = "Cancellation Confirmation"\
          ').show()

In [ ]:
52/225*100

In the small dataset has 52 churning users which makes 23% rate on this data set. Let's check over what timeframe the dataset includes sessions.

Let's use Cancellation Confirmation to create the churn label.

In [ ]:
user_log_clean = user_log_filtered 

In [ ]:
#function to create the churn label
create_churn = udf(lambda x: 1 if x =="Cancellation Confirmation" else 0, IntegerType())

user_log_clean = user_log_clean.withColumn('churn_lab', create_churn('page'))

In [ ]:
user_log_clean.take(1)

Since we only defined churn binary coding based on 'page', and that the whole dataset is based on sessionid, we are going to have users that churned but with a 0 label for all the pages they have been to that are not cancellation confirmation.

Now let's replace all the 0 of churn users to 1.



In [ ]:
# collect all the users with 1 for churn_lab
churn_userid = user_log_clean.select('userId').where(col('churn_lab')==1).groupby('userId').count()
churn_userid_list = [row['userId'] for row in churn_userid.collect()]
len(churn_userid_list)

In [ ]:
user_log_clean = user_log_clean.withColumn('churn_lab2', when((user_log_clean.userId).isin(churn_userid_list), 1).otherwise(0))

In [ ]:
user_log_clean.take(1)

In [ ]:
user_log_clean.createOrReplaceTempView("user_log_table_clean")


In [ ]:
spark.sql('SELECT userId, churn_lab, churn_lab2\
           FROM user_log_table_clean \
           WHERE churn_lab2 = 1').show()

In [ ]:

user_log_clean = user_log_clean.drop("churn_lab")
user_log_clean.take(1)

### Function for churn labeling

In [ ]:
# function for churn labeling
def create_churn_label(df):
    ''' This function takes a dataframe and creates churn labels based on the 
    churn definition: Cancellation Confirmation value in 'pages' is considered churn.
    
    Input = Spark dataframe
    
    Output = Spark dataframe with new column 'churn_lab' for each user who churned
    '''
    #labeling all 'pages' with Cancellation Confirmation
    create_churn = udf(lambda x: 1 if x =="Cancellation Confirmation" else 0, IntegerType())
    df = df.withColumn('churn_lab_temp', create_churn('page'))
    
    #extract a list of churn users from the new df
    churn_userid = df.select('userId').where(col('churn_lab_temp')==1).groupby('userId').count()
    churn_userid_list = [row['userId'] for row in churn_userid.collect()]
    
    #new column based on the churn list
    df = df.withColumn('churn_lab', when((user_log_clean.userId).isin(churn_userid_list), 1).otherwise(0))
    
    #dropping column
    df = df.drop('churn_lab_temp')
    
    return df   

In [ ]:
#let's test the function on a copy of the dataframe
user_log_test = user_log_filtered

In [ ]:
user_log_test_churn = create_churn_label(user_log_test)

In [ ]:
user_log_test_churn.take(1)

In [ ]:
user_log_test_churn.select('churn_lab').groupby('churn_lab').count().show()




### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [ ]:
#clean data sql view
user_log_clean.createOrReplaceTempView("user_log_table_clean")


In [ ]:
spark.sql('DESC user_log_table_clean').show()

- Session count

In [ ]:
# how many sessions did each userId create?
session_count_by_churn = user_log_clean.groupby('userId','churn_lab2').agg(count("*").alias("count")).toPandas()

In [ ]:
#plot of session count for each user.
plt.figure(figsize=(20, 6))
sns.barplot(data=session_count_by_churn, x='userId', y='count', hue='churn_lab2')


In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x="churn_lab2", y="count", data=session_count_by_churn, palette={0: 'blue', 1: 'orange'})

# Labels and Title
plt.xlabel('Churn Label')
plt.ylabel('Session Counts')
plt.title('Box Plot of Session Count by Churn Label')

# Show the plot
plt.show()


- artist

In [ ]:
artist_by_churn = user_log_clean.groupby('artist','churn_lab2').agg(count("*").alias("count")).toPandas().sort_values(by=['count'], ascending = False)

In [ ]:
artist_by_churn.head()

We see that the most played artist is coldplay with no churn, and over 40000 sessions did not play any songs

- 'auth'

In [ ]:
user_log_clean.select('auth').groupby('auth').count().show()

Either I remove this feature or I can explore it later with when the loggin happened.

- gender

In [ ]:
# take all users, churn and gender, drop duplicate rows by users, 
gender_by_churn = user_log_clean.dropDuplicates(['userId']).groupBy('gender','churn_lab2').agg(count('*').alias('user_count')).toPandas()
gender_by_churn.head()

In [ ]:
sns.barplot(data=gender_by_churn, x='gender', y='user_count', hue='churn_lab2')

- length

In [ ]:
user_log_clean.take(1)

In [ ]:
# look at the average session length per user and churning
columns = ['userId','length','churn_lab2']
user_log_clean_length_df = user_log_clean.select(columns)

In [ ]:
length_agg_df = user_log_clean_length_df.groupBy("userId", "churn_lab2").agg(
                spark_sum("length").alias("total_session_length"),
                spark_avg("length").alias("avg_session_length"))


In [ ]:
length_agg_df.take(10)

In [ ]:
length_agg_pd = length_agg_df.toPandas()


In [ ]:
length_agg_pd.sort_values("total_session_length").head(10)

In [ ]:
#check if I have teh same userid with 2 churnlab2
length_agg_pd.duplicated("userId").sum()

In [ ]:
length_agg_pd["userId"]=length_agg_pd["userId"].astype('category')

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x="churn_lab2", y="total_session_length", data=length_agg_pd, palette={0: 'blue', 1: 'orange'})

plt.xlabel('Churn Label')
plt.ylabel('Total Session Length')
plt.title('Box Plot of Total Session Length by Churn Label')

plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x="churn_lab2", y="avg_session_length", data=length_agg_pd, palette={0: 'blue', 1: 'orange'})

plt.xlabel('Churn Label')
plt.ylabel('Average Session Length')
plt.title('Box Plot of Average Session Length by Churn Label')

plt.show()

Even if the both churned customers and not churned have the same average session length, not churn customers have a short total session length.

In [ ]:
length_agg_pd[length_agg_pd['total_session_length'].isnull()]

level

In [ ]:
user_log_clean.dropDuplicates(['userId']).select('level').groupby('level').count().show()

this columns is related to the cancellation that I used for churn labeling so I am dropping it.

location

In [ ]:
user_log_clean.dropDuplicates(['userId']).select('location').groupby('location').count().show()

split state and city?

In [ ]:
user_log_clean.take(1)

page

In [ ]:
unique_pages_userid = (
    user_log_clean.groupBy("userId")
    .agg(
        countDistinct("page").alias("UniquePages"),
        first("churn_lab2").alias("churn_lab2")  # Assuming it's a binary label (0 or 1)
    )
)

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x="churn_lab2", y="UniquePages", data=unique_pages_userid.toPandas(), palette={0: 'blue', 1: 'orange'})

plt.xlabel('Churn Label')
plt.ylabel('Average Session Length')
plt.title('Box Plot of Unique Pages visited by Churn Label')

plt.show()

I could count the number of unique pages the user has looked at 

song

In [ ]:
unique_song_userid = (
    user_log_clean.groupBy("userId")
    .agg(
        countDistinct("song").alias("UniqueSongs"),
        first("churn_lab2").alias("churn_lab2")  # Assuming it's a binary label (0 or 1)
    )
)

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x="churn_lab2", y="UniqueSongs", data=unique_song_userid.toPandas(), palette={0: 'blue', 1: 'orange'})

plt.xlabel('Churn Label')
plt.ylabel('Number of unique songs listened to')
plt.title('Box Plot Unique Songs by Churn Label')

plt.show()

status

In [ ]:
spark.sql('SELECT status, COUNT(status) AS count\
           FROM user_log_table_clean\
           GROUP BY status').show()

dropping this column

userAgent

In [ ]:
user_log_clean.take(10)

In [ ]:
userAgent_userid = (
    user_log_clean.groupBy("userId")
    .agg(first("userAgent").alias("userAgent"),
        first("churn_lab2").alias("churn_lab2")  # Assuming it's a binary label (0 or 1)
    )
)

In [ ]:
userAgent_userid.toPandas().head(10)

In [ ]:
there is a lot of information in this column. I will extract the internet explorer platform

ts_todate

In [ ]:
user_log_clean.take(1)

In [ ]:
#isolate userid and dates
columns=['userId', 'ts_todate','registration_todate','churn_lab2']
dates_df = user_log_clean.select(columns)


In [ ]:
dates_df= dates_df.withColumn("RegistrationDate", col("registration_todate").cast("date"))
dates_df = dates_df.withColumn("SessionDate", col("ts_todate").cast("date"))


In [ ]:
dates_df.take(1)

In [ ]:
dates_df = dates_df.withColumn("NumDaysSinceRegistration", datediff(col("SessionDate"), col("RegistrationDate")))


In [ ]:
dates_df.select('NumDaysSinceRegistration','UserId','churn_lab2').toPandas()

In [ ]:
dates_df.filter(col("userid") == 24).toPandas()

is cancellation confirmation page the last date that a chrun user has used the sparkify?


In [ ]:
churn_users_df = user_log_clean.filter(col('churn_lab')==1)
churn_users_df.take(2)

In [ ]:
churnuser_page_cancelled = churn_users_df.filter(col('page') =='Cancellation Confirmation')

In [ ]:
churnuser_page_cancelled.select(["userId","ts_todate"]).orderBy(col("ts_todate").desc()).show()

In [ ]:

user_log_clean.select(["userId","ts_todate"]).orderBy(col("ts_todate").desc()).show()

registration_todate

using the registration and ts timestamps, I will calculate the number of days between the registration day and the session ts.
then I will create one col for each user with the largest number of days, the number of day where there was the largest number of sessions

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [ ]:
#let's take a look at the schema again


- select features to keep
- define what features need to be engineered and what to do
-- timestamp

#create udf functions using lambda
function_name = udf(lambda x: action(, x), IntegerType())

#to create new columns
df.withColumn("newcolname",function(df.oldcolname))

In [ ]:
user_log_clean.take(1)

In [ ]:
def prepare_data(df):
    '''This function takes a the spark dataframe and executes the following main steps:
    - checking and removing missing data in userId
    - drops all features that won't be used in the model
    - churn labeling
    - reformat timestamp to dates
    - from here, aggregation by userid
    - create new features: avg_song_length, avg_session_length, duration_registered, session_count, count_by_page_type
    - 
    
    Input: spark dataframe
    Output: clean spark dataframe including one row per userId and new features'''
    
    #feature drop
    'level'
    #cols_to_drop = ("firstname","lastname","method","auth","level")
    #df = df.drop(*cols)
    
    #missing data
    
    #churn labeling
    #use function created earlier
    
    
    
    #new features and aggregation by feature
    #allows to aggregate many features in one step
  #  agg_df = spark_df.groupBy("User_ID").agg(
  #                            mean("Feature1").alias("Avg_Feature1"),
  #                            sum("Feature1").alias("Sum_Feature1"),
  #                            max("Feature2").alias("Max_Feature2"),
   #                           first("gender").alias("gender"),
   #                           first("userAgent").alias("userAgent"),
    #                          first("location").alias("location"),
    #                          spark_sum("length").alias("total_session_length"),
    #                          spark_avg("length").alias("avg_session_length"),
    #                          first("churn_lab").alias("churn_label"),
    #                          first("level").alias("level"),
    )

   
    
    
    
    
    return clean_df

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [ ]:
# splitting data
train, test = df.randomSplit(weights=[0.8,0.2], seed=200)


In [ ]:
# give the model accuracy
lr.Model2.summary.accuracy

In [ ]:
#pipeline - create a variable for each model and then use the function Pipeline


model1 =
model2 = 

pipeline = Pipeline(stages=[model,model2])


In [ ]:
plrModel = pipeline.fit(train) #trains the model

In [ ]:
test_predictions = plrModel.transform(test) #predicts

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.